In [29]:
from bloqade.qasm2.emit import QASM2 as QASM2_target # the QASM2 target
from bloqade.qasm2.parse.lowering import QASM2

from bloqade import qasm2
from multiply_U3 import prod_U3s

import numpy as np

path=  "storage/tempcirc.qasm"


In [43]:
np.random.seed(0)


def merge(segment):
    seg = segment.copy()[::-1]
    # print(segment)
    params = seg[-1][2]
    for i in range(len(seg) -2, -1, -1):
        alpha, beta, gamma, _ = map(float, prod_U3s(*seg[i][2], *params))
        params = (alpha, beta, gamma)
    
    # print(params)
    # raise ValueError("STOP")

    return params





with open(path, "r") as f:
    original_lines = f.readlines()
    lines = [line.strip().replace(",", "").replace(";", "") for line in original_lines.copy()]
    lines = lines[3:]

    qubits = set()
    # for line in lines:

    for line in lines:
        elements = line.split()
        if len(elements) == 4:
            qubits.add(elements[-1])

    # print(qubits)

    # print(lines)
    what_to_do_with_lines = ["keep"] * len(lines)
    for qubit in qubits:
        gate_applications = []
        for i, line in enumerate(lines):
            if qubit in line:
                elements = line.split()
                # print(elements[0])
                if elements[0].startswith("U("):
                    theta = float(elements[0].split("(")[1])
                    phi = float(elements[1])
                    lam = float(elements[2].split(")")[0])
                    # params = [float(x.replace("(", "").replace(")", "")) for x in elements]
                    gate_applications.append((i, "U", (theta, phi, lam)))
                elif elements[0] == "cz":
                    gate_applications.append((i, "cz", -1))
                else:
                    raise ValueError(f"Unknown gate type: {elements[0]}")
                

        segment = []

        for i in range(len(gate_applications)):
            if gate_applications[i][1] == "U":
                segment.append(gate_applications[i])
            elif gate_applications[i][1] == "cz":
                if len(segment) > 1:
                    new_params = merge(segment)
                    what_to_do_with_lines[segment[0][0]] = ("replace", new_params)
                    for j in range(1, len(segment)):
                        what_to_do_with_lines[segment[j][0]] = "remove"
                segment = []

        if len(segment) > 1:
            new_params = merge(segment)
            what_to_do_with_lines[segment[0][0]] = ("replace", new_params)
            for j in range(1, len(segment)):
                what_to_do_with_lines[segment[j][0]] = "remove"

            
        
        # print(*gate_applications, sep="\n")
# print("WHAT TO DO WITH LINES")
# print(*what_to_do_with_lines, sep="\n")

# print(lines[-1])

new_lines = original_lines[3:]
# print(new_lines)
# print(*what_to_do_with_lines, sep="\n")

print(what_to_do_with_lines)

for i in range(len(what_to_do_with_lines)):
    if isinstance(what_to_do_with_lines[i], tuple):
        print(what_to_do_with_lines[i])
        if what_to_do_with_lines[i][0] == "replace":
            print("hey")
            new_lines[i] = f"U({what_to_do_with_lines[i][1][0]}, {what_to_do_with_lines[i][1][1]}, {what_to_do_with_lines[i][1][2]}) {lines[i].split()[-1]};\n"
            print(what_to_do_with_lines[i])
            print(new_lines[i])
        else:
            raise ValueError(f"Unknown action: {what_to_do_with_lines[i][0]}")
print(new_lines)
new_lines = [line for i, line in enumerate(new_lines) if what_to_do_with_lines[i] != "remove"]
new_lines = original_lines[:3] + new_lines
print(new_lines)

program = "".join(new_lines)



with open("storage/result_circuit.qasm", "w") as f:
    f.write(program)
    # for i, line in enumerate(lines):

    
    

[('replace', (3.141592924764994, 5.7109079911354925, 2.569315024923469)), 'remove', 'keep', ('replace', (7.0685834747005005, 7.853981627640236, -1.570796337438145)), 'keep', 'remove', 'remove', 'remove', 'keep', ('replace', (-0.7853981560922453, 1.5707963068578958, 10.995574283069885)), ('replace', (3.1415934195503903, 2.5019699277751313, 2.5019698567485427)), 'remove', 'remove', 'remove', 'keep', ('replace', (5.890486209343438, 4.712388983028415, 1.5707963067592896)), 'keep', 'remove', 'remove', 'remove', 'keep', ('replace', (4.712388991597563, 0.39269907517876396, -5.611934289901784e-09)), 'keep', 'remove', 'remove', 'remove', 'remove', 'keep', ('replace', (5.497787149080817, 4.712388978246653, 1.570796318070194)), 'keep', 'remove', 'remove', 'remove', 'keep', ('replace', (1.5707963284183435, 3.9269908116154246, 3.1415926359192214)), ('replace', (-1.5707963321226752, 3.141592650448059, 3.1415926459870493)), 'remove', 'remove', 'remove']
('replace', (3.141592924764994, 5.7109079911354

In [44]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
import numpy as np

with open ("assets/baseline/1.qasm", "r") as f:
    baseline = f.read()

with open("storage/tempcirc.qasm", "r") as f:
    unshortened = f.read()

qiskit_circuit = QuantumCircuit.from_qasm_str(program)
qiskit_circuit_baseline = QuantumCircuit.from_qasm_str(baseline)
qiskit_circuit_unshortened = QuantumCircuit.from_qasm_str(unshortened)


oper = Operator(qiskit_circuit)
oper_baseline = Operator(qiskit_circuit_baseline)
oper_unshortened = Operator(qiskit_circuit_unshortened)


print(np.linalg.norm(oper_baseline - np.exp(1j * 2) * oper))

4.367956002754722


In [45]:
min_norm = np.inf
for phi in np.linspace(0, 2*np.pi, 100):

    min_norm = min(np.linalg.norm(oper - np.exp(1j * phi) * oper_baseline), min_norm)

print(min_norm)

# phi = 0
# print(np.linalg.norm(oper - np.exp(1j * phi) * oper_baseline))

0.056340617031295316


In [5]:
print(np.allclose(oper_baseline, oper_unshortened))

True


In [42]:
qiskit_circuit.draw()

»
var_0_0: ────────────────────────────────────────────────────────────────»
                                                   ┌──────────┐          »
var_0_1: ─────────────────────────────────■────────┤ U(0,0,π) ├────────■─»
         ┌──────────────────────────────┐ │ ┌──────┴──────────┴──────┐ │ »
var_0_2: ┤ U(-7.4472e-09,5.0656,4.3592) ├─■─┤ U(0.7854,4.7124,7.854) ├─■─»
         └──────────────────────────────┘   └────────────────────────┘   »
«                                               ┌──────────┐          »
«var_0_0: ────────────────────────────■─────────┤ U(0,0,π) ├────────■─»
«          ┌────────────────────────┐ │         └──────────┘        │ »
«var_0_1: ─┤ U(6.2832,2.898,3.3851) ├─┼─────────────────────────────┼─»
«         ┌┴────────────────────────┤ │ ┌─────────────────────────┐ │ »
«var_0_2: ┤ U(7.0686,7.854,-1.5708) ├─■─┤ U(6.6759,4.7124,1.5708) ├─■─»
«         └─────────────────────────┘   └─────────────────────────┘   »
«                   ┌──────────┐                     ┌──────────┐           »
«var_0_0: ──────────┤ U(0,0,π) ├───────────■─────────┤ U(0,0,π) ├─────────■─»
«                   └──────────┘           │ ┌───────┴──────────┴───────┐ │ »
«var_0_1: ─────────────────────────────────■─┤ U(11.781,-4.7124,4.7124) ├─■─»
«         ┌──────────────────────────────┐   └──────────────────────────┘   »
«var_0_2: ┤ U(4.7124,-1.5033e-08,0.3927) ├──────────────────────────────────»
«         └──────────────────────────────┘                                  »
«         ┌─────────────────────────┐
«var_0_0: ┤ U(1.5708,3.1416,3.1416) ├
«         └┬────────────────────────┤
«var_0_1: ─┤ U(1.5708,3.1416,3.927) ├
«          └────────────────────────┘
«var_0_2: ───────────────────────────
«

In [55]:
qiskit_circuit_unshortened.draw()

»
var_0_0: ───────────────────────────────────────────────────────────»
                                          ┌──────────┐              »
var_0_1: ──────────────────────────────■──┤ U(0,0,π) ├──────────────»
         ┌────────────┐┌─────────────┐ │ ┌┴──────────┴┐┌───────────┐»
var_0_2: ┤ U(π/2,0,π) ├┤ U(π/2,0,2π) ├─■─┤ U(π/2,π,π) ├┤ U(0,0,2π) ├»
         └────────────┘└─────────────┘   └────────────┘└───────────┘»
«                                                                        »
«var_0_0: ───────────────────────────────────────────────────────────────»
«                                             ┌──────────┐ ┌────────────┐»
«var_0_1: ─────────────────────────────────■──┤ U(0,0,π) ├─┤ U(π/2,0,π) ├»
«         ┌───────────────┐┌─────────────┐ │ ┌┴──────────┴┐├───────────┬┘»
«var_0_2: ┤ U(0,5π/4,π/2) ├┤ U(π/2,0,2π) ├─■─┤ U(π/2,π,π) ├┤ U(0,0,2π) ├─»
«         └───────────────┘└─────────────┘   └────────────┘└───────────┘ »
«                                             ┌──────────┐              »
«var_0_0: ─────────────────────────────────■──┤ U(0,0,π) ├──────────────»
«          ┌─────────────┐                 │  └──────────┘              »
«var_0_1: ─┤ U(π/2,0,2π) ├─────────────────┼────────────────────────────»
«         ┌┴─────────────┴┐┌─────────────┐ │ ┌────────────┐┌───────────┐»
«var_0_2: ┤ U(0,7π/4,π/2) ├┤ U(π/2,0,2π) ├─■─┤ U(π/2,π,π) ├┤ U(0,0,2π) ├»
«         └───────────────┘└─────────────┘   └────────────┘└───────────┘»
«                                              ┌──────────┐              »
«var_0_0: ──────────────────────────────────■──┤ U(0,0,π) ├───────■──────»
«                                           │  └──────────┘       │      »
«var_0_1: ──────────────────────────────────┼─────────────────────■──────»
«         ┌────────────────┐┌─────────────┐ │ ┌────────────┐┌───────────┐»
«var_0_2: ┤ U(0,11π/8,π/2) ├┤ U(π/2,0,2π) ├─■─┤ U(π/2,π,π) ├┤ U(0,0,2π) ├»
«         └────────────────┘└─────────────┘   └────────────┘└───────────┘»
«            ┌──────────┐                                                   »
«var_0_0: ───┤ U(0,0,π) ├─────────────────────────────────────────────────■─»
«           ┌┴──────────┴┐  ┌───────────┐┌───────────────┐┌─────────────┐ │ »
«var_0_1: ──┤ U(π/2,π,π) ├──┤ U(0,0,2π) ├┤ U(0,5π/4,π/2) ├┤ U(π/2,0,2π) ├─■─»
«         ┌─┴────────────┴─┐└───────────┘└───────────────┘└─────────────┘   »
«var_0_2: ┤ U(0,13π/8,π/2) ├────────────────────────────────────────────────»
«         └────────────────┘                                                »
«          ┌──────────┐ ┌────────────┐                 
«var_0_0: ─┤ U(0,0,π) ├─┤ U(π/2,0,π) ├─────────────────
«         ┌┴──────────┴┐├───────────┬┘┌───────────────┐
«var_0_1: ┤ U(π/2,π,π) ├┤ U(0,0,2π) ├─┤ U(0,7π/4,π/2) ├
«         └────────────┘└───────────┘ └───────────────┘
«var_0_2: ─────────────────────────────────────────────
«

In [7]:
from qiskit.circuit.library import UGate


ggate = UGate(0.5, 0.5, 0.5)
print(ggate.to_matrix())

[[ 0.96891242+0.j         -0.2171174 -0.11861178j]
 [ 0.2171174 +0.11861178j  0.52350562+0.81531169j]]


In [49]:
print(oper - oper_unshortened)

Operator([[-0.35355339+4.17228019e-15j, -0.13529903+3.26640741e-01j,
            0.45710678+2.50000000e-01j,  0.13529903+3.26640741e-01j,
           -0.35355339+1.18910452e-15j, -0.13529903+3.26640741e-01j,
           -0.95710678+2.50000000e-01j,  0.13529903+3.26640741e-01j],
          [-0.35355339+6.45700524e-15j,  0.84240581-3.26640741e-01j,
           -0.25      +2.50000000e-01j, -0.13529903-3.26640741e-01j,
           -0.35355339+3.47382956e-15j, -0.57180776-3.26640741e-01j,
           -0.25      +2.50000000e-01j, -0.13529903-3.26640741e-01j],
          [ 0.35355339-2.56069383e-15j, -0.32664074-1.35299025e-01j,
            0.25      -2.50000000e-01j,  0.32664074-1.35299025e-01j,
            0.35355339-7.90709917e-15j, -0.32664074-1.35299025e-01j,
            0.25      -2.50000000e-01j,  0.32664074-1.35299025e-01j],
          [-0.35355339+1.11476565e-14j,  0.32664074+1.35299025e-01j,
            0.25      -2.50000000e-01j,  0.38046604+1.35299025e-01j,
           -0.35355339+8.085976